In [ ]:
# ============================================================================
# CHUNK 1: LIBRARY IMPORTS AND INITIAL SETUP
# ============================================================================

import nest_asyncio
nest_asyncio.apply()

from ib_insync import *
import pandas as pd
import requests
from datetime import datetime, timedelta
from twelvedata import TDClient

# 📌 Insert your API keys
td = TDClient(apikey="your_twelvedata_api_key")
FINNHUB_API_KEY = "your_finnhub_api_key"

print("CHUNK 1 ready.")


In [ ]:
# ============================================================================
# CHUNK 2: CONNECTION TO INTERACTIVE BROKERS
# ============================================================================

ib = IB()

try:
    ib.connect('127.0.0.1', 7497, clientId=6)
    print("Connected to Interactive Brokers.")
except Exception as e:
    print(f"Connection error: {e}")


In [ ]:
# ============================================================================
# CHUNK 3: FETCH EARNINGS CALENDAR FROM FINNHUB
# ============================================================================

def get_filtered_earnings_tickers():
    """
    Gets list of tickers with earnings today or yesterday from Finnhub API.
    
    Returns:
        list: tickers with announced earnings
    """
    today = datetime.now().strftime("%Y-%m-%d")
    yesterday = (datetime.now() - timedelta(days=1)).strftime("%Y-%m-%d")

    url = f"https://finnhub.io/api/v1/calendar/earnings?from={yesterday}&to={today}&token={FINNHUB_API_KEY}"

    try:
        resp = requests.get(url, timeout=20)
        data = resp.json()

        if not data or 'earningsCalendar' not in data:
            print("No earnings calendar found or bad response format:", data)
            return []

        tickers = set()
        bmo_count = amc_count = unknown_count = 0

        print("Earnings from Finnhub:")
        for item in data['earningsCalendar']:
            symbol = item.get("symbol")
            hour = (item.get("hour") or "").lower()
            
            print(f"  - {symbol} @ {hour}")
            
            if hour == "bmo":
                bmo_count += 1
            elif hour == "amc":
                amc_count += 1
            else:
                unknown_count += 1
            
            if symbol:
                tickers.add(symbol)

        print(f"Found {len(tickers)} earnings tickers.")
        print(f"Premarket (BMO): {bmo_count}")
        print(f"Aftermarket (AMC): {amc_count}")
        print(f"No hour info: {unknown_count}")

        return sorted(tickers)

    except Exception as e:
        print(f"Error fetching earnings from Finnhub: {e}")
        return []

print("CHUNK 3 ready.")


In [ ]:
# ============================================================================
# CHUNK 4: FLOAT AND SHORT INTEREST FILTER (FAST - ~0.5s)
# ============================================================================

def get_float_and_short_interest(ticker):
    """
    Gets float and short interest data from Finnhub.
    
    Criteria:
    - Float < 50M shares (low float = higher volatility)
    - Short Interest tracked (no minimum requirement)
    
    Args:
        ticker (str): stock symbol
        
    Returns:
        tuple: (float_shares, short_interest_pct) or (None, None)
    """
    try:
        url = f"https://finnhub.io/api/v1/stock/metric?symbol={ticker}&metric=all&token={FINNHUB_API_KEY}"
        resp = requests.get(url, timeout=10)
        data = resp.json()
        
        if not data or 'metric' not in data:
            return None, None
        
        metrics = data['metric']
        float_shares = metrics.get('sharesOutstanding', None)
        short_interest_pct = metrics.get('shortInterest%Float', 0) or 0
        
        if float_shares is None:
            print(f"No float data for {ticker}")
            return None, None
        
        print(f"{ticker} - Float: {float_shares:.1f}M, Short Interest: {short_interest_pct:.1f}%")
        
        if float_shares < 50:
            return float_shares, short_interest_pct
        else:
            print(f"{ticker} float too high: {float_shares:.1f}M")
            return None, None
            
    except Exception as e:
        print(f"Float/Short Interest check failed for {ticker}: {e}")
        return None, None

print("CHUNK 4 ready.")


In [ ]:
# ============================================================================
# CHUNK 5: MARKET CAP AND IPO FILTER (FAST - ~0.5s)
# ============================================================================

def check_market_cap_and_ipo(ticker):
    """
    Checks market cap and IPO year.
    
    Criteria:
    - Market Cap: $500M - $20B
    - IPO within last 15 years
    
    Args:
        ticker (str): stock symbol
        
    Returns:
        tuple: (market_cap, ipo_year) or (None, None)
    """
    try:
        contract = Stock(ticker, 'SMART', 'USD')
        details = ib.reqContractDetails(contract)
        
        if not details:
            return None, None

        market_cap = getattr(details[0].summary, "marketCap", None)

        desc = getattr(details[0].contract, "description", "") or ""
        ipo_year = None
        
        if "IPO" in desc:
            try:
                ipo_year = int(desc.split('IPO ')[-1].split()[0])
            except Exception:
                ipo_year = None

        current_year = datetime.now().year

        within_cap = (market_cap is not None and 500_000_000 <= market_cap <= 20_000_000_000)
        within_ipo = (ipo_year is not None and ipo_year >= current_year - 15)

        if within_cap and within_ipo:
            return market_cap, ipo_year
        else:
            return None, None
            
    except Exception as e:
        print(f"Market Cap/IPO check failed for {ticker}: {e}")
        return None, None

print("CHUNK 5 ready.")


In [ ]:
# ============================================================================
# CHUNK 6: REVENUE GROWTH FILTER (FAST - ~0.5s)
# ============================================================================

def get_sales_growth_quarters(ticker):
    """
    Checks quarterly revenue growth.
    
    Criteria:
    - Q1 >= 80% OR (Q1 >= 25% AND Q2 >= 25%)
    
    Args:
        ticker (str): stock symbol
        
    Returns:
        tuple: (q1_growth, q2_growth) or (None, None)
    """
    try:
        data = td.growth_estimates(symbol=ticker).as_json()
        
        if not isinstance(data, dict) or not data.get("data"):
            return None, None

        g = data["data"][0] or {}
        
        q1 = float(g.get("revenueGrowthQ1", 0) or 0)
        q2 = float(g.get("revenueGrowthQ2", 0) or 0)

        if q1 >= 80 or (q1 >= 25 and q2 >= 25):
            return q1, q2
        else:
            return None, None
            
    except Exception as e:
        print(f"Revenue growth check failed for {ticker}: {e}")
        return None, None

print("CHUNK 6 ready.")


In [ ]:
# ============================================================================
# CHUNK 7: GAP / VOLUME / PRICE FILTER (MEDIUM - ~3-4s, NO RVOL YET)
# ============================================================================

def check_gap_volume_price_basic(ticker):
    """
    Checks gap (vs yesterday close), pre-market volume, and price.
    Does NOT calculate RVol yet (that's done separately at the end).
    
    Criteria:
    - Gap > 4% (live pre-market price vs yesterday's close)
    - Pre-market Volume >= 100k shares
    - Price > $5
    
    Args:
        ticker (str): stock symbol
        
    Returns:
        dict or None: {gap_pct, volume, price, contract} if passes filters
    """
    try:
        contract = Stock(ticker, 'SMART', 'USD')
        ib.qualifyContracts(contract)

        # Get yesterday's close
        bars = ib.reqHistoricalData(
            contract,
            endDateTime='',
            durationStr='2 D',
            barSizeSetting='1 day',
            whatToShow='TRADES',
            useRTH=True,
            formatDate=1
        )
        
        if len(bars) < 1:
            print(f"Not enough historical data for {ticker}")
            return None

        prev_close = bars[-1].close
        
        # Get LIVE pre-market price (at 9:25 AM)
        ticker_data = ib.reqTickers(contract)[0]
        current_price = ticker_data.marketPrice()
        
        if pd.isna(current_price) or current_price == 0:
            current_price = ticker_data.last
        
        if pd.isna(current_price) or current_price == 0:
            print(f"No valid price data for {ticker}")
            return None
        
        # Calculate gap percentage (ignores after-hours)
        gap_pct = ((current_price - prev_close) / prev_close) * 100 if prev_close else 0.0

        # FILTER 1: Price > $5
        if current_price <= 5:
            print(f"Price too low for {ticker}: ${current_price:.2f}")
            return None

        # FILTER 2: Gap > 4%
        if gap_pct <= 4:
            print(f"{ticker} gap too small: {gap_pct:.2f}%")
            return None

        # Get today's pre-market volume
        try:
            intraday_bars = ib.reqHistoricalData(
                contract,
                endDateTime='',
                durationStr='1 D',
                barSizeSetting='1 min',
                whatToShow='TRADES',
                useRTH=False,
                formatDate=1
            )
            
            premarket_volume = 0
            for bar in intraday_bars:
                hour = bar.date.hour
                minute = bar.date.minute
                
                # Pre-market: 4:00 AM to 9:29 AM
                if (hour >= 4 and hour < 9) or (hour == 9 and minute < 30):
                    premarket_volume += bar.volume
            
            print(f"{ticker} - Pre-market volume: {premarket_volume:,}")
            
        except Exception as e:
            print(f"Could not get pre-market volume for {ticker}: {e}")
            return None

        # FILTER 3: Pre-market volume >= 100k
        if premarket_volume < 100_000:
            print(f"{ticker} pre-market volume too low: {premarket_volume:,}")
            return None

        # Return basic data + contract (needed later for RVol calculation)
        return {
            "gap_pct": round(gap_pct, 2), 
            "volume": premarket_volume, 
            "price": current_price,
            "contract": contract  # Save for RVol calculation later
        }
            
    except Exception as e:
        print(f"GAP/VOLUME/PRICE check failed for {ticker}: {e}")
        return None

print("CHUNK 7 ready.")


In [ ]:
# ============================================================================
# CHUNK 8: RELATIVE VOLUME CALCULATION (SLOW - ~10-15s)
# Called ONLY after all other filters pass
# ============================================================================

def calculate_relative_volume(ticker, contract, current_premarket_volume):
    """
    Calculate relative volume: today's pre-market volume vs 20-day average.
    
    Criteria:
    - RVol >= 1.2x
    
    Args:
        ticker (str): stock symbol
        contract: IB contract object
        current_premarket_volume (int): today's pre-market volume
        
    Returns:
        float or None: RVol multiplier if >= 1.2x, else None
    """
    try:
        print(f"Calculating RVol for {ticker} (this may take 10-15 seconds)...")
        
        # Get last 20 days of 1-min bars
        bars = ib.reqHistoricalData(
            contract,
            endDateTime='',
            durationStr='20 D',
            barSizeSetting='1 min',
            whatToShow='TRADES',
            useRTH=False,
            formatDate=1
        )
        
        # Group by date and sum pre-market volume for each day
        daily_premarket_volumes = {}
        
        for bar in bars:
            bar_date = bar.date.date()
            hour = bar.date.hour
            minute = bar.date.minute
            
            # Pre-market: 4:00 AM to 9:29 AM ET
            if (hour >= 4 and hour < 9) or (hour == 9 and minute < 30):
                if bar_date not in daily_premarket_volumes:
                    daily_premarket_volumes[bar_date] = 0
                daily_premarket_volumes[bar_date] += bar.volume
        
        if not daily_premarket_volumes:
            print(f"No historical pre-market data for {ticker}")
            return None
        
        # Calculate average
        avg_volume = sum(daily_premarket_volumes.values()) / len(daily_premarket_volumes)
        
        # Calculate RVol
        rvol = current_premarket_volume / avg_volume if avg_volume > 0 else 0
        
        print(f"{ticker} - Avg Pre-market Vol (20d): {avg_volume:,.0f}")
        print(f"{ticker} - Relative Volume: {rvol:.2f}x")
        
        # FILTER: RVol >= 1.2x
        if rvol >= 1.2:
            return round(rvol, 2)
        else:
            print(f"{ticker} RVol too low: {rvol:.2f}x (need >= 1.2x)")
            return None
        
    except Exception as e:
        print(f"RVol calculation failed for {ticker}: {e}")
        return None

print("CHUNK 8 ready.")


In [ ]:
# ============================================================================
# CHUNK 9: OPTIMIZED SCREENER EXECUTION
# RVol calculated ONLY after all other filters pass
# ============================================================================

final_candidates = []

# STEP 1: Get earnings tickers
tickers = get_filtered_earnings_tickers()

print(f"\n\n{'='*80}")
print(f"STARTING OPTIMIZED SCREENING PROCESS")
print(f"Total tickers to screen: {len(tickers)}")
print(f"{'='*80}\n")

# STEP 2: Apply FAST filters first (Float, Market Cap, Revenue)
for ticker in tickers:
    print(f"\n{'='*60}")
    print(f"Checking {ticker}...")
    print(f"{'='*60}")

    # FAST FILTER 1: Float & Short Interest (~0.5s)
    float_data = get_float_and_short_interest(ticker)
    if not float_data or None in float_data:
        print(f"❌ {ticker} failed Float filter")
        continue

    # FAST FILTER 2: Market Cap/IPO (~0.5s)
    cap_ipo = check_market_cap_and_ipo(ticker)
    if not cap_ipo or None in cap_ipo:
        print(f"❌ {ticker} failed Market Cap/IPO filter")
        continue

    # FAST FILTER 3: Revenue Growth (~0.5s)
    growth = get_sales_growth_quarters(ticker)
    if not growth or None in growth:
        print(f"❌ {ticker} failed Revenue Growth filter")
        continue

    # MEDIUM FILTER 4: Gap/Volume/Price (~3-4s, NO RVol yet)
    gap_data = check_gap_volume_price_basic(ticker)
    if not gap_data:
        print(f"❌ {ticker} failed Gap/Volume/Price filter")
        continue

    # 🎯 TICKER PASSED ALL FAST FILTERS!
    print(f"\n🎯 {ticker} passed all fast filters!")
    print(f"Now calculating RVol (slow operation)...")
    
    # SLOW FILTER 5: Relative Volume (~10-15s) - ONLY NOW
    rvol = calculate_relative_volume(
        ticker, 
        gap_data["contract"], 
        gap_data["volume"]
    )
    
    if rvol is None:
        print(f"❌ {ticker} failed RVol filter")
        continue

    # ✅ SUCCESS: Ticker passed ALL filters including RVol
    print(f"\n✅✅✅ {ticker} PASSED ALL FILTERS! ✅✅✅")
    
    final_candidates.append({
        "ticker": ticker,
        "price": gap_data["price"],
        "gap_pct": gap_data["gap_pct"],
        "premarket_volume": gap_data["volume"],
        "relative_volume": rvol,
        "float_shares_M": float_data[0],
        "short_interest_pct": float_data[1],
        "market_cap": cap_ipo[0],
        "ipo_year": cap_ipo[1],
        "revenue_growth_Q1": growth[0],
        "revenue_growth_Q2": growth[1]
    })

# STEP 3: Display and save results
print(f"\n\n{'='*80}")
print(f"FINAL RESULTS: {len(final_candidates)} stocks passed all filters")
print(f"{'='*80}\n")

if final_candidates:
    df = pd.DataFrame(final_candidates)
    
    # Sort by gap % descending
    df = df.sort_values('gap_pct', ascending=False)
    
    print("Selected tickers (sorted by gap):")
    print(", ".join(df['ticker'].tolist()))
    
    print("\nDetailed results:")
    print(df.to_string(index=False))
    
    df.to_csv("peads_filtered_stocks_optimized.csv", index=False)
    print("\nSaved to 'peads_filtered_stocks_optimized.csv'")
else:
    print("No stocks met all criteria today.")
